In [9]:
from datafeed import *

In [28]:
import sqlite3 as sql
import config
import sys
sys.path.append("../")
sys.path.append("C:\\Users\\X_X\\Desktop\\TraddingBot\\back_end\\")
sys.path.append('C:\\Users\\X_X\\Desktop\\TraddingBot\\back-end\\signals')
sys.path.append('C:\\Users\\X_X\\Desktop\\TraddingBot\\back-end\\signals\\backtrader_signal')
sys.path.append('C:\\Users\\X_X\\Desktop\\TraddingBot\\back-end\\backtrader')


import backtrader as bt
import sqlite3 as sql
import config
import pandas as pd

from EMA import EMA

def fetchAsPandas(query, *params):
    list_params = []
    for param in params:
        list_params.append(param)
    
    conn = sql.connect(config.DB_URL, detect_types=sql.PARSE_DECLTYPES)
    cur = conn.cursor()
    query_result = cur.execute(query, list_params)
    cols = [column[0] for column in query_result.description]
    results= pd.DataFrame.from_records(data = query_result.fetchall(), columns = cols)
    conn.close()
    
    return results

def fetch_coin_price_as_pandas(symbol, interval):
    symbol = symbol.upper()
    select_query = '''SELECT open_time as datetime, open, high, low, close, volume 
                   from coin_pair a inner join coin_price b on (a.id = b.id_coin_pair)
                   where symbol = ? and interval = ? '''
    result = fetchAsPandas(select_query, symbol, interval)
    result = result.set_index('datetime')
    
    return result

def create_coin_price_dataFeed(symbol, interval, **kwargs):
    data = fetch_coin_price_as_pandas(symbol, interval)
    name = symbol.upper()+ '_' + interval.upper()
    
    return bt.feeds.PandasData(dataname = data, name = name, **kwargs)

def get_all_dataFeed(interval, **kwargs):
    
    conn = sql.connect(config.DB_URL)
    conn.row_factory = sql.Row
    cur = conn.cursor()
    
    select_query = "SELECT DISTINCT symbol FROM coin_pair where interval = ?"
    rows = cur.execute(select_query, (interval, ))
    result_list = []
    
    
    for row in rows:
        new_dataFeed = create_coin_price_dataFeed(row['symbol'], interval, **kwargs)
        result_list.append(new_dataFeed)
    
    conn.close()
    
    return result_list

In [36]:
import time 
symbol ='btcusdt'
interval = '1d'
start_time_1 = time.clock()

data_feed = create_coin_price_dataFeed('BTCUSDT', '1d')
print(f'Time execution 2: {time.clock() - start_time_1}')
# Create a data feed
start_time_3 = time.clock()
cerebro = bt.Cerebro() 
cerebro.adddata(data_feed)  # Add the data feed
cerebro.addstrategy(EMA)  # Add the trading strategy
result = cerebro.run(stdstats=False)  # run it all
check = None
print(f'Time execution 3: {time.clock() - start_time_3}')

start_time_4 = time.clock()
if (result[0].signal[0] == 1):
    check = True
else :
    check = False
print(f'Time execution 4: {time.clock() - start_time_4}')


print(f'Time execution 1: {time.clock() - start_time_1}')

Time execution 2: 0.02127450000079989
Time execution 3: 0.303019700000732
Time execution 4: 9.130000034929253e-05
Time execution 1: 0.32481279999956314


In [24]:
import time 
symbol ='btcusdt'
interval = '1d'
start_time = time.clock()
create_coin_price_dataFeed(symbol, interval)
print(f'Time execution 1: {time.clock() - start_time}')

Time execution 1: 0.024110599999403348


In [6]:
print(row['open_time'])

2017-08-17 04:00:00


In [15]:
def create_coin_price_dataFeed(symbol, interval, **kwargs):
    data = fetch_coin_price_as_pandas(symbol, interval)
    name = symbol.upper()+ '_' + interval.upper()
    
    return bt.feeds.PandasData(dataname = data, name = name, **kwargs)
